## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the __`country_club`__ database.

<br>
1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable,
please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

### Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [0]:
%sql 
drop database if exists country_club cascade;
create database country_club;
show databases;

databaseName
country_club
default


### Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you. 
To get started, first upload the three CSV files to the DBFS as depicted in the following figure

![](https://i.imgur.com/QcCruBr.png)


Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [0]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [0]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [0]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [0]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [0]:
%sql
select * from bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03T11:00:00.000+0000,2
1,4,1,2012-07-03T08:00:00.000+0000,2
2,6,0,2012-07-03T18:00:00.000+0000,2


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [0]:
%sql
select name from facilities where membercost > 0

name
Tennis Court 1
Tennis Court 2
Massage Room 1
Massage Room 2
Squash Court


####  Q2: How many facilities do not charge a fee to members?

In [0]:
%sql
select count(*) from facilities where membercost = 0

count(1)
4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [0]:
%sql
select facid, name, membercost, monthlymaintenance from facilities
where membercost > 0 and membercost < 0.2 * monthlymaintenance

facid,name,membercost,monthlymaintenance
0,Tennis Court 1,5.0,200
1,Tennis Court 2,5.0,200
4,Massage Room 1,9.9,3000
5,Massage Room 2,9.9,3000
6,Squash Court,3.5,80


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [0]:
%sql
select * from facilities where facid = 1 or facid = 5

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [0]:
%sql
select name, 
  case
    when monthlymaintenance > 100 then 'expensive'
    else 'cheap'
    end monthlymaintenance
from facilities

name,monthlymaintenance
Tennis Court 1,expensive
Tennis Court 2,expensive
Badminton Court,cheap
Table Tennis,cheap
Massage Room 1,expensive
Massage Room 2,expensive
Squash Court,cheap
Snooker Table,cheap
Pool Table,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [0]:
%sql
select firstname, surname from members inner join bookings on members.memid = bookings.memid

firstname,surname
Darren,Smith
Darren,Smith
GUEST,GUEST
Darren,Smith
Darren,Smith
Darren,Smith
Tracy,Smith
Tracy,Smith
Tim,Rownam
GUEST,GUEST


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [0]:
%sql
select first(facilities.name) as facility, concat(members.firstname, ' ', members.surname) as member from members 
inner join bookings on members.memid = bookings.memid
inner join facilities on facilities.facid = bookings.facid
where facilities.name like "%Tennis Court%"
group by member
order by member

facility,member
Tennis Court 1,Anne Baker
Tennis Court 2,Burton Tracy
Tennis Court 1,Charles Owen
Tennis Court 2,Darren Smith
Tennis Court 1,David Farrell
Tennis Court 2,David Jones
Tennis Court 1,David Pinker
Tennis Court 1,Douglas Jones
Tennis Court 1,Erica Crumpet
Tennis Court 2,Florence Bader


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [0]:
%sql
select facilities.name,  concat(members.firstname, ' ', members.surname) as member, 
case
  when bookings.memid = 0 then facilities.guestcost * bookings.slots
  else facilities.membercost * bookings.slots
  end totalcost
  from bookings 
inner join facilities on facilities.facid = bookings.facid
inner join members on members.memid = bookings.memid
where date(starttime) = '2012-09-14' and 
  case 
    when bookings.memid = 0 then facilities.guestcost * bookings.slots > 30
    else facilities.membercost * bookings.slots > 30
  end
order by totalcost

name,member,totalcost
Squash Court,GUEST GUEST,35.0
Squash Court,GUEST GUEST,35.0
Massage Room 1,Jemima Farrell,39.6
Squash Court,GUEST GUEST,70.0
Tennis Court 1,GUEST GUEST,75.0
Tennis Court 1,GUEST GUEST,75.0
Tennis Court 2,GUEST GUEST,75.0
Tennis Court 2,GUEST GUEST,150.0
Massage Room 1,GUEST GUEST,160.0
Massage Room 1,GUEST GUEST,160.0


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [0]:
%sql
select * from (
select facilities.name,  concat(members.firstname, ' ', members.surname) as member, 
case
  when bookings.memid = 0 then facilities.guestcost * bookings.slots
  else facilities.membercost * bookings.slots
  end totalcost
  from bookings 
inner join facilities on facilities.facid = bookings.facid
inner join members on members.memid = bookings.memid
where date(starttime) = '2012-09-14'
)
where totalcost > 30
order by totalcost

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03T11:00:00.000+0000,2
1,4,1,2012-07-03T08:00:00.000+0000,2
2,6,0,2012-07-03T18:00:00.000+0000,2
3,7,1,2012-07-03T19:00:00.000+0000,2
4,8,1,2012-07-03T10:00:00.000+0000,1
5,8,1,2012-07-03T15:00:00.000+0000,1
6,0,2,2012-07-04T09:00:00.000+0000,3
7,0,2,2012-07-04T15:00:00.000+0000,3
8,4,3,2012-07-04T13:30:00.000+0000,2
9,4,0,2012-07-04T15:00:00.000+0000,2


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [0]:
%sql
select * from (
select facilities.name,
sum(
case
  when bookings.memid = 0 then facilities.guestcost * bookings.slots
  else facilities.membercost * bookings.slots
  end
  ) as totalrevenue
from facilities
inner join bookings on bookings.facid = facilities.facid
group by facilities.name
)
where totalrevenue < 1000

name,totalrevenue
Pool Table,270.0
Table Tennis,180.0
Snooker Table,240.0
